In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from langdetect import DetectorFactory, detect
from langdetect.lang_detect_exception import LangDetectException
DetectorFactory.seed = 0

def is_english(text):
    try:
        if detect(text) != "en":
            return False
    except LangDetectException:
        return False
    return True

def scrape_tweets(keyword_search:str, since_date:str, until_date:str):
    tweets = []
    for j, tweet in enumerate(sntwitter.TwitterSearchScraper(keyword_search+' since:'+since_date+' until:'+until_date).get_items()):
#        if j>100000:
#           break
        tweets.append([tweet.date, tweet.id, tweet.content, tweet.user.username])
    tweets_df = pd.DataFrame(tweets, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
    tweets_df['language'] =  tweets_df['Text'].apply(is_english)
    tweets_df = tweets_df[tweets_df['language'] == True].reset_index()
    return tweets_df

In [23]:
tweets_df = scrape_tweets("transplant", "2022-01-01", "2022-07-07")

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=transplant+since%3A2022-01-01+until%3A2022-07-07&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaCwKuV9KCP4CkWgIC8obTqrPAqEnEVn_6HARWAiXoYB0RFRkFVTFQ1ARX0qgEVAAA%3D&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/search/adaptive.json?include_profile_

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=transplant+since%3A2022-01-01+until%3A2022-07-07&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaCwKuV9KCP4CkWgIC8obTqrPAqEnEVn_6HARWAiXoYB0RFRkFVTFQ1ARX0qgEVAAA%3D&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel failed, giving up.

In [24]:
tweets_df.head()

,index,Datetime,Tweet Id,Text,Username,language
0,14,2022-07-06 23:28:09+00:00,1544825559498244102,@TheDemocrats @MeidasTouch They will never giv...,cathy_venus,True
1,16,2022-07-06 23:20:20+00:00,1544823591463751683,"@karisferg I hope you get the chance, it’s cool",cathy_venus,True
2,40,2022-07-06 22:04:02+00:00,1544804388761370631,CMV- a (white tee) par 3 should be reachable f...,FoGOLF,True
3,52,2022-07-06 20:39:03+00:00,1544783004828139522,We are excited to announce the detailed agenda...,EHDIconference,True
4,53,2022-07-06 20:38:01+00:00,1544782745993445381,We are excited to announce the detailed agenda...,InfantHearing,True


In [20]:


from transformers import pipeline

classifier = pipeline("zero-shot-classification")

classifier(

    tweets_df['Text'][1],

    candidate_labels=["transplant", "maribavir", "health care", "burden", "care", "financial"],

)

No model was supplied, defaulted to roberta-large-mnli (https://huggingface.co/roberta-large-mnli)
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at roberta-large-mnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


{'sequence': '@karisferg I hope you get the chance, it’s cool',
 'labels': ['care',
  'burden',
  'financial',
  'transplant',
  'maribavir',
  'health care'],
 'scores': [0.37932509183883667,
  0.32932835817337036,
  0.10205322504043579,
  0.09276972711086273,
  0.05484664812684059,
  0.04167698696255684]}

In [22]:
tweets_df['Text'][3]

'We are excited to announce the detailed agenda for the 2022 CMV Public Health and Policy Conference is now posted. View the agenda at https://t.co/DiXF1Q6AY2. Register by July 15 to save on costs. We look forward to seeing you in Ottawa for this one-of-a-kind conference! https://t.co/HQiNw6lLF8'